In [1]:
pip install taipy

     |████████████████████████████████| 2.6 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 11.4 MB/s ta 0:00:01
     |████████████████████████████████| 12.0 MB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 7.7 MB/s  eta 0:00:01
     |████████████████████████████████| 45 kB 6.7 MB/s  eta 0:00:01
     |████████████████████████████████| 1.9 MB 34.5 MB/s eta 0:00:01
     |████████████████████████████████| 232 kB 23.5 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 17.7 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 5.8 MB/s  eta 0:00:01
     |████████████████████████████████| 73 kB 6.3 MB/s  eta 0:00:01
     |████████████████████████████████| 489 kB 20.2 MB/s eta 0:00:01
     |████████████████████████████████| 525 kB 23.6 MB/s eta 0:00:01
     |████████████████████████████████| 149 kB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 

     |████████████████████████████████| 340 kB 43.5 MB/s eta 0:00:01
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 1.1.1
    Uninstalling MarkupSafe-1.1.1:
      Successfully uninstalled MarkupSafe-1.1.1
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.2
    Uninstalling Werkzeug-2.0.2:
      Successfully uninstalled Werkzeug-2.0.2
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
  Attempting uninstall: packaging
    Found existing installation: packaging 21.0
    Uninstalling packaging-21.0:
      Successfully uninstalled packaging-21.0
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.4
    Uninstalling pandas-1.3.4:
      Successfully uninstalled 

In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [5]:
from platform import python_version

print(python_version())

3.9.7


In [6]:
from taipy import Gui
Gui(page="# Getting started with *Taipy*").run(dark_mode=False)


 * Server starting on http://127.0.0.1:5000


In [7]:
import pandas as pd

def get_data(path_to_csv: str):
    # pandas.read_csv() returns a pd.DataFrame
    dataset = pd.read_csv(path_to_csv)
    dataset["Date"] = pd.to_datetime(dataset["Date"])
    return dataset

# Read the dataframe
path_to_csv = "dataset.csv"
dataset = get_data(path_to_csv)

In [8]:
from taipy import Gui

dataset = get_data(path_to_csv)

# Initial value
n_week = 10

# Definition of the page
page = """
# Getting started with Taipy

Week number: *<|{n_week}|>*

Interact with this slider to change the week number:
<|{n_week}|slider|min=1|max=52|>

## Dataset:

Display the last three months of data:
<|{dataset[9000:]}|chart|type=bar|x=Date|y=Value|height=100%|>

<|{dataset}|table|height=400px|width=95%|>
"""

# Create a Gui object with our page content
Gui(page=page).run(dark_mode=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


In [10]:
# Select the week based on the slider value
dataset_week = dataset[dataset["Date"].dt.isocalendar().week == n_week]

page = """
# Getting started with Taipy

Select week: *<|{n_week}|>*

<|{n_week}|slider|min=1|max=52|>

<|{dataset_week}|chart|type=bar|x=Date|y=Value|height=100%|width=100%|>
"""

# on_change is the function that is called when any variable is changed
def on_change(state, var_name: str, var_value):
    if var_name == "n_week":
        # Update the dataset when the slider is moved
        state.dataset_week = dataset[dataset["Date"].dt.isocalendar().week == var_value]

Gui(page=page).run(dark_mode=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


In [18]:
import datetime as dt
import pandas as pd

from taipy import Config, Scope

## Input Data Nodes
initial_dataset_cfg = Config.configure_data_node(id="initial_dataset",
                                                 storage_type="csv",
                                                 path=path_to_csv,
                                                 scope=Scope.GLOBAL)

# We assume the current day is the 26th of July 2021.
# This day can be changed to simulate multiple executions of scenarios on different days
day_cfg = Config.configure_data_node(id="day", default_data=dt.datetime(2021, 7, 26))

n_predictions_cfg = Config.configure_data_node(id="n_predictions", default_data=40)

max_capacity_cfg = Config.configure_data_node(id="max_capacity", default_data=200)

In [19]:
import datetime as dt
import pandas as pd

from taipy import Config, Scope

## Input Data Nodes
initial_dataset_cfg = Config.configure_data_node(id="initial_dataset",
                                                 storage_type="csv",
                                                 path=path_to_csv,
                                                 scope=Scope.GLOBAL)

# We assume the current day is the 26th of July 2021.
# This day can be changed to simulate multiple executions of scenarios on different days
day_cfg = Config.configure_data_node(id="day", default_data=dt.datetime(2021, 7, 26))

n_predictions_cfg = Config.configure_data_node(id="n_predictions", default_data=40)


In [20]:
## Remaining Data Nodes
cleaned_dataset_cfg = Config.configure_data_node(id="cleaned_dataset",
                                             cacheable=True,
                                             validity_period=dt.timedelta(days=1),
                                             scope=Scope.GLOBAL) 

predictions_cfg = Config.configure_data_node(id="predictions", scope=Scope.PIPELINE)

def clean_data(initial_dataset: pd.DataFrame):
    print("     Cleaning data")
    # Convert the date column to datetime
    initial_dataset["Date"] = pd.to_datetime(initial_dataset["Date"])
    cleaned_dataset = initial_dataset.copy()
    return cleaned_dataset


def predict_baseline(cleaned_dataset: pd.DataFrame, n_predictions: int, day: dt.datetime, max_capacity: int):
    print("     Predicting baseline")
    # Select the train data
    train_dataset = cleaned_dataset[cleaned_dataset["Date"] < day]

    predictions = train_dataset["Value"][-n_predictions:].reset_index(drop=True)
    predictions = predictions.apply(lambda x: min(x, max_capacity))
    return predictions

clean_data_task_cfg = Config.configure_task(id="clean_data",
                                            function=clean_data,
                                            input=initial_dataset_cfg,
                                            output=cleaned_dataset_cfg)

predict_baseline_task_cfg = Config.configure_task(id="predict_baseline",
                                                  function=predict_baseline,
                                                  input=[cleaned_dataset_cfg, n_predictions_cfg, day_cfg, max_capacity_cfg],
                                                  output=predictions_cfg)


In [49]:
# Create the first pipeline configuration
baseline_pipeline_cfg = Config.configure_pipeline(id="baseline",
                                                  task_configs=[clean_data_task_cfg, predict_baseline_task_cfg])

import taipy as tp

# Create the pipeline
baseline_pipeline = tp.create_pipeline(baseline_pipeline_cfg)
# Submit the pipeline (Execution)
tp.submit(baseline_pipeline)

# Read output data from the pipeline
baseline_predictions = baseline_pipeline.predictions.read()
print("Predictions of baseline algorithm\n", baseline_predictions)

     Cleaning data
[2023-01-31 15:35:16,215][Taipy][INFO] job JOB_clean_data_37a2611a-72f8-401b-9df6-8196d5dd385e is completed.
     Predicting baseline
[2023-01-31 15:35:16,246][Taipy][INFO] job JOB_predict_baseline_132ac1dd-a1b5-4c7d-bc2c-96eab7785eff is completed.
Predictions of baseline algorithm
 0     128
1     108
2     106
3      99
4      84
5     107
6     118
7      93
8     121
9      98
10    108
11     99
12    114
13     90
14    129
15     87
16    102
17    120
18    110
19    166
20    109
21    109
22     70
23    132
24     57
25    140
26    137
27     98
28    119
29    130
30    191
31    143
32    132
33    159
34    184
35    101
36    119
37     62
38     98
39      8
Name: Value, dtype: int64


In [50]:
import numpy as np
import pandas as pd

# Initialize the "predictions" dataset
predictions_dataset = pd.DataFrame({"Date":[dt.datetime(2021, 6, 1)], "Historical values":[np.NaN], "Predicted values":[np.NaN]})

# Add a button and a chart for our predictions
pipeline_page = page + """
Press <|predict|button|on_action=predict|> to predict with default parameters (30 predictions) and June 1st as day.

<|{predictions_dataset}|chart|x=Date|y[1]=Historical values|type[1]=bar|y[2]=Predicted values|type[2]=scatter|height=80%|width=100%|>
"""

In [51]:
def predict(state):
    print("'Predict' button clicked")
    pipeline = create_and_submit_pipeline()
    update_predictions_dataset(state, pipeline)


def create_and_submit_pipeline():
    print("Execution of pipeline...")
    # Create the pipeline from the pipeline config
    pipeline = tp.create_pipeline(baseline_pipeline_cfg)
    # Submit the pipeline (Execution)
    tp.submit(pipeline)
    return pipeline

In [52]:
def create_predictions_dataset(pipeline):
    print("Creating predictions dataset...")
    # Read data from the pipeline
    predictions = pipeline.predictions.read()
    day = pipeline.day.read()
    n_predictions = pipeline.n_predictions.read()
    cleaned_data = pipeline.cleaned_dataset.read()

    # Set arbitrarily the time window for the chart as 5 times the number of predictions
    window = 5 * n_predictions

    # Create the historical dataset that will be displayed
    new_length = len(cleaned_data[cleaned_data["Date"] < day]) + n_predictions
    temp_df = cleaned_data[:new_length]
    temp_df = temp_df[-window:].reset_index(drop=True)

    # Create the series that will be used in the concat
    historical_values = pd.Series(temp_df["Value"], name="Historical values")
    predicted_values = pd.Series([np.NaN]*len(temp_df), name="Predicted values")
    predicted_values[-len(predictions):] = predictions

    # Create the predictions dataset
    # Columns : [Date, Historical values, Predicted values]
    return pd.concat([temp_df["Date"], historical_values, predicted_values], axis=1)

In [53]:
def update_predictions_dataset(state, pipeline):
    print("Updating predictions dataset...")
    state.predictions_dataset = create_predictions_dataset(pipeline)


In [54]:
Gui(page=pipeline_page).run(dark_mode=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


In [58]:
# For the sake of clarity, we have used an AutoRegressive model rather than a pure ML model such as:
# Random Forest, Linear Regression, LSTM, etc   
from statsmodels.tsa.ar_model import AutoReg

# This is the function that will be used by the task
def predict_ml(cleaned_dataset: pd.DataFrame, n_predictions: int, day: dt.datetime, max_capacity: int):
    print("     Predicting with ML")
    # Select the train data
    train_dataset = cleaned_dataset[cleaned_dataset["Date"] < day]

    # Fit the AutoRegressive model
    model = AutoReg(train_dataset["Value"], lags=7).fit()

    # Get the n_predictions forecasts
    predictions = model.forecast(n_predictions).reset_index(drop=True)
    predictions = predictions.apply(lambda x: min(x, max_capacity))
    return predictions



In [60]:
# Create the task configuration of the predict_ml function.
## We use the same input and ouput as the previous predict_baseline task but we change the funtion
predict_ml_task_cfg = Config.configure_task(id="predict_ml",
                                            function=predict_ml,
                                            input=[cleaned_dataset_cfg, n_predictions_cfg, day_cfg, max_capacity_cfg],
                                            output=predictions_cfg)


In [61]:
# Create the new ml pipeline that will clean and predict with the ml model
ml_pipeline_cfg = Config.configure_pipeline(id="ml", task_configs=[clean_data_task_cfg, predict_ml_task_cfg])


In [62]:
# Set the list of pipelines names
# It will be used in a selector of pipelines
pipeline_selector = ["baseline", "ml"]
selected_pipeline = pipeline_selector[0]
scenario_page = page + """
Select the pipeline
<|{selected_pipeline}|selector|lov={pipeline_selector}|> <|Update chart|button|on_action=update_chart|>

<|{predictions_dataset}|chart|x=Date|y[1]=Historical values|type[1]=bar|y[2]=Predicted values|type[2]=scatter|height=80%|width=100%|>
"""


In [63]:
# Configure our scenario which is our business problem.
scenario_cfg = Config.configure_scenario(id="scenario", pipeline_configs=[baseline_pipeline_cfg, ml_pipeline_cfg])


In [64]:
def create_scenario():
    print("Creating scenario...")
    scenario = tp.create_scenario(scenario_cfg)
    scenario = submit_scenario(scenario)
    return scenario

def submit_scenario(scenario):
    print("Submitting scenario...")
    tp.submit(scenario)
    return scenario

def update_chart(state):
    print("'Update chart' button clicked")
    # Select the right pipeline
    pipeline = scenario.pipelines[state.selected_pipeline]

    # Update the chart based on this pipeline
    # It is the same function as created before in step_5
    update_predictions_dataset(state, pipeline)


In [65]:
# Delete all entities
Config.configure_global_app(clean_entities_enabled=True)
tp.clean_all_entities()

True

In [66]:
# Creation of our first scenario
scenario = create_scenario()
Gui(page=scenario_page).run(dark_mode=False) 


Creating scenario...
Submitting scenario...
     Cleaning data
[2023-01-31 15:38:39,102][Taipy][INFO] job JOB_clean_data_6d16c7f6-4e2e-4c83-aa02-3e93a13159cd is completed.
     Predicting baseline
[2023-01-31 15:38:39,120][Taipy][INFO] job JOB_predict_baseline_53b18120-41e4-49af-b242-0e6d58bd06fc is completed.
[2023-01-31 15:38:39,144][Taipy][INFO] job JOB_clean_data_4b300d03-0996-47a1-b6cd-20837680693b is skipped.
     Predicting with ML
[2023-01-31 15:38:39,253][Taipy][INFO] job JOB_predict_ml_a81861c0-d072-46ad-a99d-68e124f29970 is completed.


/Users/apple/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
/Users/apple/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/Users/apple/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:141: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(index, pd.Int64Index) and np.all(np.diff(index) == 1):
/Users/apple/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:143: FutureWarning: pandas.In

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


In [38]:
# Initial variables
## Initial variables for the scenario   
day = dt.datetime(2021, 7, 26)
n_predictions = 40
max_capacity = 200

In [67]:
page_scenario_manager = page + """
# Change your scenario

**Prediction date**\n\n <|{day}|date|not with_time|>

**Max capacity**\n\n <|{max_capacity}|number|>

**Number of predictions**\n\n<|{n_predictions}|number|>

<|Save changes|button|on_action={submit_scenario}|>

Select the pipeline
<|{selected_pipeline}|selector|lov={pipeline_selector}|> <|Update chart|button|on_action={update_chart}|>

<|{predictions_dataset}|chart|x=Date|y[1]=Historical values|type[1]=bar|y[2]=Predicted values|type[2]=scatter|height=80%|width=100%|>
"""

In [68]:
def create_scenario():
    global selected_scenario

    print("Creating scenario...")
    scenario = tp.create_scenario(scenario_cfg)

    selected_scenario = scenario.id

    tp.submit(scenario)


In [69]:
def submit_scenario(state):
    print("Submitting scenario...")
    # Get the selected scenario: in this current step a single scenario is created then modified here.
    scenario = tp.get(selected_scenario)

    # Conversion to the right format
    state_day = dt.datetime(state.day.year, state.day.month, state.day.day)

    # Change the default parameters by writing in the datanodes
    scenario.day.write(state_day)
    scenario.n_predictions.write(int(state.n_predictions))
    scenario.max_capacity.write(int(state.max_capacity))

    # Execute the pipelines/code
    tp.submit(scenario)

    # Update the chart when we change the scenario
    update_chart(state)

In [70]:
def update_chart(state):
    # Select the right scenario and pipeline
    scenario = tp.get(selected_scenario)
    pipeline = scenario.pipelines[state.selected_pipeline]
    # Update the chart based on this pipeline
    update_predictions_dataset(state, pipeline)


global selected_scenario
# Creation of a single scenario
create_scenario()
Gui(page=page_scenario_manager).run(dark_mode=False)


Creating scenario...
[2023-01-31 15:39:27,078][Taipy][INFO] job JOB_clean_data_2be10bca-c006-48c8-bc66-244049c9b9fb is skipped.
     Predicting baseline
[2023-01-31 15:39:27,102][Taipy][INFO] job JOB_predict_baseline_467c8a41-723e-40cf-ada2-c561d61aab63 is completed.
[2023-01-31 15:39:27,126][Taipy][INFO] job JOB_clean_data_3f647273-fcff-4b61-b797-1cfc55ffc749 is skipped.
     Predicting with ML
[2023-01-31 15:39:27,153][Taipy][INFO] job JOB_predict_ml_06d02462-67f3-4ef8-b2a5-4aad5c7d4b0e is completed.
Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


/Users/apple/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
/Users/apple/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/Users/apple/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:141: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(index, pd.Int64Index) and np.all(np.diff(index) == 1):
/Users/apple/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:143: FutureWarning: pandas.In

In [71]:
# Get all the scenarios already created
all_scenarios = tp.get_scenarios()

# Delete the scenarios that don't have a name attribute
# All the scenarios of the previous steps do not have an associated name so they will be deleted,
# this will not be the case for those created by this step
[tp.delete(scenario.id) for scenario in all_scenarios if scenario.name is None]

# Initial variable for the scenario selector
# The list of possible values (lov) for the scenario selector is a list of tuples (scenario_id, scenario_name),
# but the selected_scenario is just used to retrieve the scenario id and what gets displayed is the name of the scenario.
scenario_selector = [(scenario.id, scenario.name) for scenario in tp.get_scenarios()]
selected_scenario = None

In [72]:
scenario_manager_page = page + """
# Create your scenario

**Prediction date**\n\n <|{day}|date|not with_time|>

**Max capacity**\n\n <|{max_capacity}|number|>

**Number of predictions**\n\n<|{n_predictions}|number|>

<|Create new scenario|button|on_action=create_scenario|>

## Scenario 
<|{selected_scenario}|selector|lov={scenario_selector}|dropdown|>

## Display the pipeline
<|{selected_pipeline}|selector|lov={pipeline_selector}|>

<|{predictions_dataset}|chart|x=Date|y[1]=Historical values|type[1]=bar|y[2]=Predicted values|type[2]=scatter|height=80%|width=100%|>
"""


def create_name_for_scenario(state)->str:
    name = f"Scenario ({state.day.strftime('%A, %d %b')}; {state.max_capacity}; {state.n_predictions})"

    # Change the name if it is the same as some scenarios
    if name in [s[1] for s in state.scenario_selector]:
        name += f" ({len(state.scenario_selector)})"
    return name


def update_chart(state):
    # Now, the selected_scenario comes from the state, it is interactive
    scenario = tp.get(state.selected_scenario[0])
    pipeline = scenario.pipelines[state.selected_pipeline]
    update_predictions_dataset(state, pipeline)


# Change the create_scenario function in order to change the default parameters
# and allow the creation of multiple scenarios
def create_scenario(state):
    print("Execution of scenario...")
    # Extra information for the scenario
    creation_date = state.day
    name = create_name_for_scenario(state)
    # Create a scenario
    scenario = tp.create_scenario(scenario_cfg,creation_date=creation_date, name=name)

    state.selected_scenario = (scenario.id, name)
    # Submit the scenario that is currently selected
    submit_scenario(state)


def submit_scenario(state):
    print("Submitting scenario...")
    # Get the currently selected scenario
    scenario = tp.get(state.selected_scenario[0])

    # Conversion to the right format (change?)
    day = dt.datetime(state.day.year, state.day.month, state.day.day) 

    # Change the default parameters by writing in the Data Nodes
    scenario.day.write(day)
    scenario.n_predictions.write(int(state.n_predictions))
    scenario.max_capacity.write(int(state.max_capacity))
    scenario.creation_date = state.day


    # Execute the scenario
    tp.submit(scenario)

    # Update the scenario selector and the scenario that is currently selected
    update_scenario_selector(state, scenario) # change list to scenario

    # Update the chart directly
    update_chart(state)

In [73]:
from taipy import Gui

def on_menu():
    print('Menu function called')

Gui(page="<|menu|label=Menu|lov={['Data Visualization', 'Scenario Manager']}|on_action=on_menu|>").run()

<|part|render={bool_variable}|
Text
Or visual elements...
|>

<|layout|columns=1 1 1|
Button in first column <|Press|button|>

Second column

Third column
|>


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


In [74]:
# Our first page is the original page
# (with the slider and the chart that displays a week of the historical data)
page_data_visualization = page

In [75]:
# Second page: create scenarios and display results
page_scenario_manager = """
# Create your scenario

<|layout|columns=1 1 1 1|
<|
**Prediction date**\n\n <|{day}|date|not with_time|>
|>

<|
**Max capacity**\n\n <|{max_capacity}|number|>
|>

<|
**Number of predictions**\n\n<|{n_predictions}|number|>
|>

<|
<br/>\n <|Create new scenario|button|on_action=create_scenario|>
|>
|>

<|part|render={len(scenario_selector) > 0}|
<|layout|columns=1 1|
<|
## Scenario \n <|{selected_scenario}|selector|lov={scenario_selector}|dropdown|>
|>

<|
## Display the pipeline \n <|{selected_pipeline}|selector|lov={pipeline_selector}|dropdown|>
|>
|>

<|{predictions_dataset}|chart|x=Date|y[1]=Historical values|type[1]=bar|y[2]=Predicted values|type[2]=scatter|height=80%|width=100%|>
|>
"""


In [76]:
# Create a menu with our pages
multi_pages = """
<|menu|label=Menu|lov={["Data Visualization", "Scenario Manager"]}|on_action=on_menu|>

<|part|render={page=="Data Visualization"}|""" + page_data_visualization + """|>
<|part|render={page=="Scenario Manager"}|""" + page_scenario_manager + """|>
"""


# The initial page is the "Data Visualization" page
page = "Data Visualization"
def on_menu(state, var_name: str, fct: str, var_value: list):
    # Change the value of the state.page variable in order to render the correct page
    state.page = var_value["args"][0]


Gui(page=multi_pages).run(dark_mode=False)


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


In [77]:
from taipy import Config, Frequency

# Create scenarios each week and compare them
scenario_daily_cfg = Config.configure_scenario(id="scenario",
                                           pipeline_configs=[baseline_pipeline_cfg, ml_pipeline_cfg],
                                           frequency=Frequency.DAILY)

In [78]:
# Delete all entities
Config.configure_global_app(clean_entities_enabled=True)
tp.clean_all_entities()

True

In [79]:
# Change the inital scenario selector to see which scenarios are primary
scenario_selector = [(scenario.id, ("*" if scenario.is_primary else "") + scenario.name) for scenario in tp.get_scenarios()]

# Redefine update_scenario_selector to add "*" in the display name when the scnario is primary
def update_scenario_selector(state, scenario):
    print("Updating scenario selector...")
    # Create the scenario name for the scenario selector
    # This name changes dependind whether the scenario is primary or not
    scenario_name = ("*" if scenario.is_primary else "") + scenario.name
    print(scenario_name)
    # Update the scenario selector
    state.scenario_selector += [(scenario.id, scenario_name)]

In [80]:
# Change the create_scenario function to create a scenario with the selected frequency
def create_scenario(state):
    print("Execution of scenario...")
    # Extra information for scenario
    creation_date = state.day
    name = create_name_for_scenario(state)

    # Create a scenario with the week cycle
    scenario = tp.create_scenario(scenario_daily_cfg, creation_date=creation_date, name=name)

    state.selected_scenario = (scenario.id, name)

    # Change the scenario that is currently selected
    submit_scenario(state)


In [81]:
selected_scenario_is_primary = None

def make_primary(state):
    print("Making the current scenario primary...")
    scenario = tp.get(state.selected_scenario[0])
    # Take the current scenario primary
    tp.set_primary(scenario)

    # Update the scenario selector accordingly
    state.scenario_selector = [(scenario.id, ("*" if scenario.is_primary else "") + scenario.name)
                               for scenario in tp.get_scenarios()]
    state.selected_scenario_is_primary = True


In [82]:
from taipy.gui import notify

def remove_scenario_from_selector(state, scenario: list):
    # Take all the scenarios in the selector that doesn't have the scenario.id
    state.scenario_selector = [(s[0], s[1]) for s in state.scenario_selector if s[0] != scenario.id]
    state.selected_scenario = state.scenario_selector[-1]

def delete_scenario(state):
    scenario = tp.get(state.selected_scenario[0])

    if scenario.is_primary:
        # Notify the user that primary scenarios can not be deleted
        notify(state, "info", "Cannot delete the primary scenario")
    else:
        # Delete the scenario and the related objects (datanodes, tasks, jobs,...)
        tp.delete(scenario.id)

        # Update the scenario selector accordingly
        remove_scenario_from_selector(state,scenario)

In [83]:
# Add a "Delete scenario" and a "Make primary" buttons
page_scenario_manager = """
# Create your scenario:

<|layout|columns=1 1 1 1|
<|
**Prediction date**\n\n <|{day}|date|not with_time|>
|>

<|
**Max capacity**\n\n <|{max_capacity}|number|>
|>

<|
**Number of predictions**\n\n<|{n_predictions}|number|>
|>

<|
<br/>
<br/>
<|Create new scenario|button|on_action=create_scenario|>
|>
|>


<|part|render={len(scenario_selector) > 0}|
<|layout|columns=1 1|

<|layout|columns=1 1|
<|
## Scenario \n <|{selected_scenario}|selector|lov={scenario_selector}|dropdown|>
|>

<br/>
<br/>
<br/>
<br/>
<|Delete scenario|button|on_action=delete_scenario|active={len(scenario_selector)>0}|>
<|Make primary|button|on_action=make_primary|active={not(selected_scenario_is_primary) and len(scenario_selector)>0}|>
|>


<|
## Display the pipeline \n <|{selected_pipeline}|selector|lov={pipeline_selector}|dropdown|>
|>
|>

<|{predictions_dataset}|chart|x=Date|y[1]=Historical values|type[1]=bar|y[2]=Predicted values|type[2]=scatter|height=80%|width=100%|>
|>
"""

# Redefine the multi_pages
multi_pages = """
<|menu|label=Menu|lov={["Data Visualization", "Scenario Manager"]}|on_action=on_menu|>

<|part|render={page=="Data Visualization"}|""" + page_data_visualization + """|>
<|part|render={page=="Scenario Manager"}|""" + page_scenario_manager + """|>
"""

def on_change(state, var_name: str, var_value):
    if var_name == "n_week":
        # Update the dataset when the slider is moved
        state.dataset_week = dataset[dataset["Date"].dt.isocalendar().week == var_value]

    elif var_name == "selected_pipeline" or var_name == "selected_scenario":
        # Update selected_scenario_is_primary indicating if the current scenario is primary or not
        state.selected_scenario_is_primary = tp.get(state.selected_scenario[0]).is_primary

        # Check if we can read the Data Node to update the chart
        if tp.get(state.selected_scenario[0]).predictions.read() is not None:
            update_chart(state)

In [84]:
Gui(page=multi_pages).run(dark_mode=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000
